In [1]:
import psycopg2 as pg
import pandas as pd

In [2]:
import pandas.io.sql as psql

# Get work by WP

In [40]:
connection = pg.connect("host=10.10.11.116 port=5433 dbname=bregis_lis user=lis_admin password=Bre91s_Admin")
select_wp = """
select w."text" work_name
from lab.work as w,
	 lab.workplace as wp
where wp."text" = 'SysmexXS500i'
    and w.workplace_id = wp.id
"""
# dataframe_wp = psql.read_sql(select, connection)

cursor = connection.cursor()

cursor.execute(select_wp)
wp = cursor.fetchall()
works = [w[0] for w in wp]
works

['Атипичные мононуклеары [SysmexXS500i]',
 'Флаги патологий [SysmexXS500i]',
 'Ретикулоциты [SysmexXS500i]',
 'Клинический анализ крови [SysmexXS500i]']

# Get Data

In [3]:
# connection = pg.connect("host=10.10.11.116 port=5433 dbname=bregis_lis user=lis_admin password=Bre91s_Admin")
select = """
select wr."text" result_name, wrv.*
from lab.work as w,
   lab.workresult as wr,
   lab.workresult_value as wrv
where w."text" = 'Клинический анализ крови [BС5300]'
    and wr.work_id = w.id
    and wrv.workresult_id = wr.id
"""
dataframe = psql.read_sql(select, connection)
# product_category = psql.read_sql_query('select * from product_category', connection)
dataframe['result_name'] = dataframe['result_name'].str.lower()
dataframe

C:\Users\BatrakovDR\AppData\Local\Temp\ipykernel_25988\281852555.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dataframe = psql.read_sql(select, connection)


,result_name,id,createby,createdate,updateby,updatedate,workresult_id,code,text,type_id,...,display_format_id,result_print_format_id,norm_set_id,hide_zero_value,input_order,display_order,sp_func,ext_code,status,group_num
0,нейтрофилы % (числовой),3720,1,2024-02-13 13:58:22.440894,1,2024-02-13 13:59:00.795571,3567,2042,Количественный,2039,...,2103,None,NaN,0,0,0,None,None,1,None
1,гематокрит (hct) (числовой),3719,1,2024-02-13 13:56:46.562073,1,2024-02-13 13:57:50.415380,3566,2042,Количественный,2039,...,2103,None,NaN,0,0,0,None,None,1,None
2,гематокрит (hct) (анализатор),3298,-1,2021-03-24 14:00:35.000000,1,2021-03-24 16:03:15.000000,3171,2042,Количественный (%),2039,...,2106,None,8.0,0,0,0,None,1017359,1,None
3,коэффициент больших тромбоцитов,3326,-1,2021-03-24 14:00:36.000000,1,2021-03-24 16:08:56.000000,3199,2042,Количественный (%),2039,...,2103,None,NaN,0,0,0,None,None,1,None
4,относительная ширина распределения тромбоцитов...,3325,-1,2021-03-24 14:00:36.000000,1,2021-03-24 16:09:10.000000,3198,2042,Количественный (фл),2039,...,2103,None,NaN,0,0,0,None,None,1,None
5,соэ,3324,-1,2021-03-24 14:00:36.000000,-1,2021-03-24 14:00:36.000000,3197,2042,Количественный (мм/ч),2039,...,2103,None,NaN,0,0,0,None,None,1,None
6,сегментоядерные,3323,-1,2021-03-24 14:00:36.000000,-1,2021-03-24 14:00:36.000000,3196,2042,Количественный (%),2039,...,2103,None,1.0,0,0,0,None,None,1,None
7,палочкоядерные,3322,-1,2021-03-24 14:00:35.000000,-1,2021-03-24 14:00:35.000000,3195,2042,Количественный (%),2039,...,2103,None,1.0,0,0,0,None,None,1,None
8,атипичные мононуклеары,3321,-1,2021-03-24 14:00:35.000000,1,2021-04-02 10:57:46.000000,3194,2042,Количественный (%),2039,...,2103,None,NaN,1,0,0,None,None,1,None
9,глюкоза,3320,-1,2021-03-24 14:00:35.000000,1,2021-04-02 10:57:27.000000,3193,2042,Количественный (ммоль/л),2039,...,2103,None,NaN,0,0,0,None,None,1,None


In [4]:
func_df = dataframe.dropna(subset=['func'])
func_df = func_df[['result_name', 'id', 'func', 'text']]
func_df

,result_name,id,func,text
0,нейтрофилы % (числовой),3720,IF IS_NUMBER(#3306)\nTHEN #3306\nELSE -1,Количественный
1,гематокрит (hct) (числовой),3719,IF IS_NUMBER(#3298)\nTHEN #3298\nELSE -1,Количественный


In [5]:
df = dataframe.loc[(dataframe['result_name'].isin(func_df['result_name']))]
df

,result_name,id,createby,createdate,updateby,updatedate,workresult_id,code,text,type_id,...,display_format_id,result_print_format_id,norm_set_id,hide_zero_value,input_order,display_order,sp_func,ext_code,status,group_num
0,нейтрофилы % (числовой),3720,1,2024-02-13 13:58:22.440894,1,2024-02-13 13:59:00.795571,3567,2042,Количественный,2039,...,2103,None,NaN,0,0,0,None,None,1,None
1,гематокрит (hct) (числовой),3719,1,2024-02-13 13:56:46.562073,1,2024-02-13 13:57:50.415380,3566,2042,Количественный,2039,...,2103,None,NaN,0,0,0,None,None,1,None


# Get values

In [6]:
ident_1 = '(числовой)'
ident_2 = '(анализатор)'

In [7]:
names = df['result_name'].str.replace(ident_1, '')
names

0        нейтрофилы % 
1    гематокрит (hct) 
Name: result_name, dtype: object

In [8]:
target_values = dataframe[dataframe['result_name'].str.contains(ident_1)]
analizator_values = dataframe[dataframe['result_name'].str.contains(ident_2)]

C:\Users\BatrakovDR\AppData\Local\Temp\ipykernel_25988\3204257614.py:1: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  target_values = dataframe[dataframe['result_name'].str.contains(ident_1)]
C:\Users\BatrakovDR\AppData\Local\Temp\ipykernel_25988\3204257614.py:2: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  analizator_values = dataframe[dataframe['result_name'].str.contains(ident_2)]


## Analizator values

In [9]:
analizator_values

,result_name,id,createby,createdate,updateby,updatedate,workresult_id,code,text,type_id,...,display_format_id,result_print_format_id,norm_set_id,hide_zero_value,input_order,display_order,sp_func,ext_code,status,group_num
2,гематокрит (hct) (анализатор),3298,-1,2021-03-24 14:00:35,1,2021-03-24 16:03:15,3171,2042,Количественный (%),2039,...,2106,None,8.0,0,0,0,None,1017359,1,None
23,нейтрофилы % (анализатор),3306,-1,2021-03-24 14:00:35,1,2021-03-24 16:06:40,3179,2042,Количественный (%),2039,...,2104,None,12.0,0,0,0,None,1018290,1,None


In [10]:
analizator_values_without_ident = analizator_values['result_name'].str.replace(ident_2, '')

In [11]:
analizator_values.loc[:, ('result_name')] = analizator_values_without_ident
analizator_values

,result_name,id,createby,createdate,updateby,updatedate,workresult_id,code,text,type_id,...,display_format_id,result_print_format_id,norm_set_id,hide_zero_value,input_order,display_order,sp_func,ext_code,status,group_num
2,гематокрит (hct),3298,-1,2021-03-24 14:00:35,1,2021-03-24 16:03:15,3171,2042,Количественный (%),2039,...,2106,None,8.0,0,0,0,None,1017359,1,None
23,нейтрофилы %,3306,-1,2021-03-24 14:00:35,1,2021-03-24 16:06:40,3179,2042,Количественный (%),2039,...,2104,None,12.0,0,0,0,None,1018290,1,None


## Target values

In [12]:
target_values

,result_name,id,createby,createdate,updateby,updatedate,workresult_id,code,text,type_id,...,display_format_id,result_print_format_id,norm_set_id,hide_zero_value,input_order,display_order,sp_func,ext_code,status,group_num
0,нейтрофилы % (числовой),3720,1,2024-02-13 13:58:22.440894,1,2024-02-13 13:59:00.795571,3567,2042,Количественный,2039,...,2103,None,NaN,0,0,0,None,None,1,None
1,гематокрит (hct) (числовой),3719,1,2024-02-13 13:56:46.562073,1,2024-02-13 13:57:50.415380,3566,2042,Количественный,2039,...,2103,None,NaN,0,0,0,None,None,1,None


In [13]:
target_values_without_ident = target_values['result_name'].str.replace(ident_1, '')
target_values_without_ident

0        нейтрофилы % 
1    гематокрит (hct) 
Name: result_name, dtype: object

In [14]:
target_values.loc[:, ('result_name')] = target_values_without_ident
target_values

,result_name,id,createby,createdate,updateby,updatedate,workresult_id,code,text,type_id,...,display_format_id,result_print_format_id,norm_set_id,hide_zero_value,input_order,display_order,sp_func,ext_code,status,group_num
0,нейтрофилы %,3720,1,2024-02-13 13:58:22.440894,1,2024-02-13 13:59:00.795571,3567,2042,Количественный,2039,...,2103,None,NaN,0,0,0,None,None,1,None
1,гематокрит (hct),3719,1,2024-02-13 13:56:46.562073,1,2024-02-13 13:57:50.415380,3566,2042,Количественный,2039,...,2103,None,NaN,0,0,0,None,None,1,None


# Changing formulas

In [15]:
target_names = target_values['result_name'].tolist()
target_names

['нейтрофилы % ', 'гематокрит (hct) ']

In [17]:
for name in target_names:
    idx_anal = analizator_values.loc[analizator_values['result_name'] == name]['id'].tolist()[0]
    target_func = target_values.loc[target_values['result_name'] == name]['func'].tolist()[0]
    # find all values in func
    i = 0
    value_indxs = []
    while i != -1:
        # print(target_func[sum(value_indxs)::])
        i = target_func[sum(value_indxs)+1::].find('#')
        value_indxs.append(i+1)
    # last element is bad
    value_indxs.pop(-1)
    # replace all values in func on new values
    for n in value_indxs:
        tf = target_func[:n+1+1:] + str(idx_anal) + target_func[n+1+1+4::]
        target_values.loc[target_values['result_name'] == name, 'func'] = tf

In [20]:
target_values['id']

0    3720
1    3719
Name: id, dtype: int64

# Update

In [36]:
cursor = connection.cursor()
ind = target_values.index
for i in ind:
    v = target_values.loc[i]
    table_modification = f"""UPDATE lab.workresult_value as wrv SET func = '{str(v['func'])}' WHERE id = {int(v['id'])};"""
    # print(table_modification)
    try:
        cursor.execute(table_modification)
        connection.commit()
    except:
        print('Except!')
        cursor.execute("ROLLBACK")
        connection.commit()